In [1]:
import pandas as pd
pd.set_option('display.max_rows', 20)
pd.set_option('max_colwidth', 500)

In [3]:
import difflib


In [4]:
import pandas as pd
from pyfaidx import Fasta
import difflib


hg38 = Fasta('/fast/projects/cubit/current/static_data/reference/GRCh38/hs38/hs38.fa',
               one_based_attributes=False )
hg37 = Fasta('/fast/projects/cubit/current/static_data/reference/GRCh37/hs37/hs37.fa',
               one_based_attributes=False )

file = 'input/variants/hg38/SNVs.lifted.to.hg38.all.20160109.bed.gz'
var = pd.read_csv(file, sep = '\t', header = None)

ref_hg38 = []
diffs = []
for i in range(len(var)):
    ref = hg38[var[0][i]][var[1][i]:var[2][i]].seq


    ref_int_hg38 = hg38[var[0][i]][var[1][i]-5:var[2][i]+5].seq
    p = int(var[3][i].split(':')[1])
    ref_int_hg37 = hg37[var[0][i][3:]][p-1-50:p+50].seq
    #diff= sum([i[0] != ' '  for i in difflib.ndiff(ref_int_hg38, ref_int_hg37)]) / 2
    diff = sum([int(i!=j) for i,j in zip(ref_int_hg38,ref_int_hg37)])
    diffs.append(1-diff/101)
    ref_hg38.append(ref)
    
    
ref_hg37 = var[3].str.split(':', expand = True)[2]
# alternative variants
alt= var[3].str.split(':', expand = True)[3]
var[5]=ref_hg38
var[6]=diffs
var[7] = alt
# exclude variants that are not alternative any more
new_var = var[ref_hg38!=alt]


new_var = new_var[[0,1,2,5,7,3]]
#new_var.to_csv('input/variants/SNVs.lifted.to.hg38.checked.all.20160109.bed.gz',  compression="gzip", sep = '\t', index = None, header = None)



In [26]:
#new_var[new_var[6]!=1]

In [5]:
var 

,0,1,2,3,5,6,7
0,chr1,7961858,7961859,chr1:8021919:C:G,C,0.910891,G
1,chr1,9943501,9943502,chr1:10003560:A:T,A,0.910891,T
2,chr1,9943502,9943503,chr1:10003561:C:T,C,0.910891,T
3,chr1,11023350,11023351,chr1:11083408:G:A,G,0.930693,A
4,chr1,21509426,21509427,chr1:21835920:C:T,C,0.910891,T
...,...,...,...,...,...,...,...
448,chrX,155022770,155022771,chrX:154251046:G:A,G,0.900990,A
449,chrX,155022772,155022773,chrX:154251048:A:T,A,0.910891,T
450,chrX,155022806,155022807,chrX:154251082:T:C,T,0.930693,C
451,chrX,155022808,155022809,chrX:154251084:A:C,A,0.930693,C


In [21]:
len(var[var[5].str.len()>1])-len(var)

-416

In [43]:

new_var[6].median()


0.9370629370629371

In [34]:
from scipy.spatial import distance
a = 'jmjkhk'
b = 'mjkhkj'
print(sum([int(i!=j) for i,j in zip(a,b)]))
print(sum([i[0] != ' '  for i in difflib.ndiff(a,b )])/2 )
print(distance.jaccard(a,b))

6
1.0
1.0


In [33]:
a = 'jmjkhk'
b = 'mjkhkj'

distance.jaccard(a,b)

/fast/users/nazaretl_c/work/miniconda/envs/jupyter/lib/python3.8/site-packages/scipy/spatial/distance.py:881: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  nonzero = np.bitwise_or(u != 0, v != 0)


1.0